In [2]:
import numpy as np
import pandas as pd
import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

In [3]:
path_submission = 'results/submissions/'

In [4]:
%%time
df = pd.read_csv('data/clean/train_clean.csv')
df_targets = pd.read_csv('data/clean/train_labels.csv')
df_test = pd.read_csv('data/clean/test_clean.csv')

CPU times: user 49.8 s, sys: 4.68 s, total: 54.5 s
Wall time: 54.4 s


### Experimento 1
Entrenando sólo con el mes de Junio 2015 y con un modelo para todos los productos

In [5]:
x_train = df[df['fecha_dato'] == '2015-06-28']
y_train = df_targets.loc[x_train.index]

x = x_train.drop(['fecha_dato', 'fecha_alta', 'ncodpers'], axis=1).as_matrix()
y = y_train.as_matrix()

x_test = df_test.drop(['fecha_dato', 'fecha_alta', 'ncodpers'], axis=1).as_matrix()

In [6]:
%%time
rf = RandomForestClassifier(n_jobs=4)
rf.fit(x, y)

CPU times: user 1min 47s, sys: 6.87 s, total: 1min 54s
Wall time: 34.2 s


In [7]:
%%time
probs = rf.predict_proba(x_test)
preds = rf.predict(x_test)

CPU times: user 2min 14s, sys: 4.54 s, total: 2min 19s
Wall time: 1min 3s


In [ ]:
pred_probs = np.array([pr.max(axis=1) for pr in probs]).T
pred_probs = (preds * pred_probs).argsort(axis=1)
pred_probs = np.fliplr(pred_probs)[:, :7]

In [15]:
targets = np.array(df_targets.columns.tolist())

In [150]:
%%time
final_pred = [" ".join(list(targets[p])) for p in pred_probs]

df_subm = pd.DataFrame({'ncodpers': df_test.ncodpers.values, 'added_products': final_pred})
df_subm.head()

CPU times: user 7.13 s, sys: 56 ms, total: 7.19 s
Wall time: 7.18 s


In [151]:
df_subm.head()

,added_products,ncodpers
0,ind_cco_fin_ult1 ind_recibo_ult1 ind_nom_pens_...,15889
1,ind_cco_fin_ult1 ind_recibo_ult1 ind_nom_pens_...,1170544
2,ind_cco_fin_ult1 ind_recibo_ult1 ind_nom_pens_...,1170545
3,ind_cco_fin_ult1 ind_recibo_ult1 ind_nom_pens_...,1170547
4,ind_cco_fin_ult1 ind_recibo_ult1 ind_nom_pens_...,1170548


In [ ]:
%%time
df_subm.to_csv('scripts/results/submissions/2018_07_09.csv', index=False)

### Experimento 1a
Modificación del experimento anterior, solo que la *submission* se va a hacer sólo con los productos agregados

In [14]:
ncodpers_last_month = df_test['ncodpers'].values
df_prev_month = df[df['fecha_dato'] == '2016-05-28']
ncodpers_prev_month = df_prev_month.loc[:, 'ncodpers'].values

In [153]:
df_prev_month.shape

(700444, 28)

In [15]:
df_prev_month.head()

,fecha_dato,fecha_dato_year,fecha_dato_month,fecha_dato_day,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,...,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
10094948,2016-05-28,2016,2,28,657640,1,1,1,68,2006-11-17,...,2,1,33,1,1.0,28.0,1,0.0,147347.97,1
10094949,2016-05-28,2016,2,28,657788,1,1,1,42,2006-11-18,...,2,1,62,1,1.0,28.0,1,1.0,132559.35,1
10094950,2016-05-28,2016,2,28,657795,1,1,1,44,2006-11-18,...,2,1,35,1,1.0,26.0,27,1.0,81399.57,1
10094951,2016-05-28,2016,2,28,657794,1,1,1,49,2006-11-18,...,2,1,2,1,1.0,8.0,2,1.0,102189.00,3
10094952,2016-05-28,2016,2,28,657789,1,1,1,36,2006-11-18,...,2,1,2,1,1.0,28.0,1,1.0,153725.49,3


In [16]:
df_prev_month.reset_index(drop=True, inplace=True)
df_prev_month.head()

,fecha_dato,fecha_dato_year,fecha_dato_month,fecha_dato_day,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,...,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,2016-05-28,2016,2,28,657640,1,1,1,68,2006-11-17,...,2,1,33,1,1.0,28.0,1,0.0,147347.97,1
1,2016-05-28,2016,2,28,657788,1,1,1,42,2006-11-18,...,2,1,62,1,1.0,28.0,1,1.0,132559.35,1
2,2016-05-28,2016,2,28,657795,1,1,1,44,2006-11-18,...,2,1,35,1,1.0,26.0,27,1.0,81399.57,1
3,2016-05-28,2016,2,28,657794,1,1,1,49,2006-11-18,...,2,1,2,1,1.0,8.0,2,1.0,102189.00,3
4,2016-05-28,2016,2,28,657789,1,1,1,36,2006-11-18,...,2,1,2,1,1.0,28.0,1,1.0,153725.49,3


In [154]:
ncodpers_both = list(set(ncodpers_last_month) & set(ncodpers_prev_month))

index_prev = df_prev_month[df_prev_month['ncodpers'].isin(ncodpers_both)].index
index_last = df_test[df_test['ncodpers'].isin(ncodpers_both)].index

prev_prods = df_targets.loc[index_prev].as_matrix()
pred_prods = preds[index_last, :]

both_prods = pred_prods - prev_prods
both_prods = (both_prods < 0) * 1

preds[index_last] = both_prods

pred_probs = np.array([pr.max(axis=1) for pr in probs]).T
pred_probs = (preds * pred_probs).argsort(axis=1)
pred_probs = np.fliplr(pred_probs)[:, :7]

In [160]:
final_pred = [" ".join(list(targets[p])) for p in pred_probs]

df_subm = pd.DataFrame({'ncodpers': df_test.ncodpers.values, 'added_products': final_pred})

In [161]:
df_subm.head()

,added_products,ncodpers
0,ind_recibo_ult1 ind_nom_pens_ult1 ind_aval_fin...,15889
1,ind_cco_fin_ult1 ind_recibo_ult1 ind_nom_pens_...,1170544
2,ind_cco_fin_ult1 ind_recibo_ult1 ind_nom_pens_...,1170545
3,ind_recibo_ult1 ind_nom_pens_ult1 ind_aval_fin...,1170547
4,ind_recibo_ult1 ind_nom_pens_ult1 ind_aval_fin...,1170548


In [162]:
%%time
df_subm.to_csv('scripts/results/submissions/2018_07_10_prev_prods.csv', index=False)

CPU times: user 7.49 s, sys: 116 ms, total: 7.6 s
Wall time: 9.01 s


### Experimento 1b - con ajustes del anterior

In [16]:
ncodpers_last_month = df_test['ncodpers'].values
df_prev_month = df[df['fecha_dato'] == '2016-05-28']
ncodpers_prev_month = df_prev_month.loc[:, 'ncodpers'].values

In [17]:
ncodpers_both = list(set(ncodpers_last_month) & set(ncodpers_prev_month))

index_prev = df_prev_month[df_prev_month['ncodpers'].isin(ncodpers_both)].index
index_last = df_test[df_test['ncodpers'].isin(ncodpers_both)].index

prev_prods = df_targets.loc[index_prev].as_matrix()
pred_prods = preds[index_last, :]

both_prods = pred_prods - prev_prods
both_prods = (both_prods < 0) * 1

preds[index_last] = both_prods

pred_probs = np.array([pr.max(axis=1) for pr in probs]).T
pred_probs = (preds * pred_probs).argsort(axis=1)
pred_probs = np.fliplr(pred_probs)[:, :7]

In [18]:
final_pred = [" ".join(list(targets[p])) for p in pred_probs]

df_subm = pd.DataFrame({'ncodpers': df_test.ncodpers.values, 'added_products': final_pred})

In [21]:
df_subm.head()

,added_products,ncodpers
0,ind_recibo_ult1 ind_nom_pens_ult1 ind_aval_fin...,15889
1,ind_recibo_ult1 ind_cno_fin_ult1 ind_nom_pens_...,1170544
2,ind_cco_fin_ult1 ind_recibo_ult1 ind_nom_pens_...,1170545
3,ind_recibo_ult1 ind_nom_pens_ult1 ind_aval_fin...,1170547
4,ind_recibo_ult1 ind_nom_pens_ult1 ind_aval_fin...,1170548


In [20]:
name_file = path_submission + time.strftime("%Y-%m-%d-h%H-%M-%S_") + "submission.csv"
print(name_file)
df_subm.to_csv(name_file, index=False)

results/submissions/2018-07-12-h00-47-02_submission.csv


---
### Experimento 2
Entrenamiento de un modelo para cada uno de los productos

**Nota:** Se puede hacer una comparación entre 1 y 2


In [177]:
%%time
preds = []
probs = []
for i, col in enumerate(targets):
    
    rf.fit(x, y[:, i])
    
    pd = rf.predict(x_test)
    pb = rf.predict_proba(x_test).max(axis=1)
        
    preds.append(pd)
    probs.append(pb)
    print(col)

ind_ahor_fin_ult1
ind_aval_fin_ult1
ind_cco_fin_ult1
ind_cder_fin_ult1
ind_cno_fin_ult1
ind_ctju_fin_ult1
ind_ctma_fin_ult1
ind_ctop_fin_ult1
ind_ctpp_fin_ult1
ind_deco_fin_ult1
ind_deme_fin_ult1
ind_dela_fin_ult1
ind_ecue_fin_ult1
ind_fond_fin_ult1
ind_hip_fin_ult1
ind_plan_fin_ult1
ind_pres_fin_ult1
ind_reca_fin_ult1
ind_tjcr_fin_ult1
ind_valo_fin_ult1
ind_viv_fin_ult1
ind_nomina_ult1
ind_nom_pens_ult1
ind_recibo_ult1
CPU times: user 7min 5s, sys: 8.62 s, total: 7min 14s
Wall time: 2min 15s


In [184]:
preds = np.array(preds).T
probs = np.array(probs).T

pred_probs = (preds * probs).argsort(axis=1)
pred_probs = np.fliplr(pred_probs)[:, :7]

final_pred = [" ".join(list(targets[p])) for p in pred_probs]

df_subm = pd.DataFrame({'ncodpers': df_test.ncodpers.values, 'added_products': final_pred})
df_subm.head()

In [196]:
name_file = path_submission + time.strftime("%Y-%m-%d-h%H-%M-%S_") + "submission.csv"
print(name_file)
df_subm.to_csv(name_file, index=False)

scripts/results/submissions/2018-07-10-h10-45-09_submission.csv


### Experimento 3
Entrenamiento con Julio 2015, un solo modelo GaussianNB para cada producto

In [210]:
%%time
gb = GaussianNB()
preds = []
probs = []
for i, col in enumerate(targets):
    
    gb.fit(x, y[:, i])
    
    prd = rf.predict(x_test)
    pb = rf.predict_proba(x_test).max(axis=1)
        
    preds.append(prd)
    probs.append(pb)

CPU times: user 2min 43s, sys: 6.04 s, total: 2min 49s
Wall time: 1min 6s


In [217]:
preds = np.array(preds).T
probs = np.array(probs).T

In [227]:
%%time
pred_probs = (preds * probs).argsort(axis=1)
pred_probs = np.fliplr(pred_probs)[:, :7]

final_pred = [" ".join(list(targets[p])) for p in pred_probs]

df_subm = pd.DataFrame({'ncodpers': df_test.ncodpers.values, 'added_products': final_pred})
df_subm.head()

CPU times: user 7.55 s, sys: 196 ms, total: 7.74 s
Wall time: 7.74 s


In [228]:
%%time
name_file = path_submission + time.strftime("%Y-%m-%d-h%H-%M-%S_") + "submission.csv"
print(name_file)
df_subm.to_csv(name_file, index=False)

scripts/results/submissions/2018-07-10-h11-18-47_submission.csv
CPU times: user 7.49 s, sys: 80 ms, total: 7.57 s
Wall time: 8.88 s
